In [1]:
from influxdb_client import InfluxDBClient

url = "http://localhost:8086"

# hot-wip
#token = "gCGWIFyp0f0WxPnz3w_k7N993EPqvf-4qit7H8XMs5eoPJx4LdUHNnrc371Vvd6hYAdrBcEm_k89HlIAwUk3LA=="
#org = "user"

# nob-pch
#token = "JYn7HQ4YRosmEdIAKonWE5KeMNhu28e-p1erovFlDgQPkqbkinUZ_ty-uDbL2eNvuHs_RFE142p5lSi80qTDRg=="
#org = "tclab"

# nob-nbh
token = "ycHUSYx8Tg91dl41rYndfT0El5dk6kEPZgyZPaSAcoPyf8t5khfshxIS0ISIyw_pLTBhprNvXmfbHQxBTdhrXg=="
org = "tclab"

import os 
import json

def read_(fname):
    rc = ""
    with open(fname) as f:
        rc = f.readlines()
    return "".join(rc)

def write_(tables,fname,verbose=0):
    # load json result
    output = json.loads(tables.to_json())

    ls = "\n"
    sep = "|"

    if verbose: print(f"write {os.path.basename(fname)}")

    with open(fname,"w") as f:
        if len(output) > 0:
            # header
            columns = list(output[0].keys())
            _ = [columns.remove(c) for c in ['result']]
            rc = sep.join(c for c in columns)
            f.write(f"{rc}{ls}")
            # values
            for row in output:
                rc = sep.join(str(row[c]) for c in columns)
                f.write(f"{rc}{ls}")
        else:
            rc = "NONE"
            f.write(f"{rc}{ls}")

    if verbose: print(f"wrote {len(output)} line(s) of {fname}")


In [16]:
%%time

fpath = os.path.join(os.getcwd(),"queries","2")
fnames = {fname: os.path.join(fpath,fname) for fname in os.listdir(fpath) if fname.endswith(".flux")}
fqueries = {fname: read_(fnames[fname]) for fname in fnames.keys()}

fname = "234b.flux"

with InfluxDBClient(url=url, token=token, org=org, debug=False, timeout=240000) as client:
    
    query_api = client.query_api()        
    query = fqueries[fname]

    tables = query_api.query(org=org, query=query)
    write_(tables, fname=os.path.join(fpath,"%s.txt"%(fname)),verbose=1)


write 234b.flux.txt
wrote 24 line(s) of c:\Develop\vsc\tclab\idb\queries\2\234b.flux.txt
CPU times: total: 15.6 ms
Wall time: 73.4 ms
